In [13]:
#Libraries 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import os

# Import functions for data preprocessing & data preparation
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer, LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import string
from string import punctuation
import nltk
import re

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bader\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [7]:
data = pd.read_csv('comments.csv')
data

,Unnamed: 0,Comment,Likes,Time,user,UserLink
0,0,Love how Dr. Fate's design looks and how cool ...,12K,9 days ago,NaN,NaN
1,1,I can’t get over how good everything looks. Dr...,2.6K,7 days ago,NaN,NaN
2,2,Really hoping that this can save DC's movie un...,814,7 days ago,NaN,NaN
3,3,U cant deny how good this looks.Now if they ca...,91,6 days ago,NaN,NaN
4,4,"From this trailer, I have a feeling that this ...",367,7 days ago (edited),NaN,NaN
...,...,...,...,...,...,...
275,275,I want to see this. It may be one of his most ...,0,6 days ago (edited),NaN,NaN
276,276,wow thats very amazing. I can't wait to see.,0,6 days ago,NaN,NaN
277,277,Doctor Fate is why i'm watching,5,1 day ago,NaN,NaN
278,278,This looks fire. DC looks like they stepping t...,0,7 days ago,NaN,NaN


In [8]:
data1=data.drop(['Unnamed: 0','Likes','Time','user','UserLink'],axis=1)
data1

,Comment
0,Love how Dr. Fate's design looks and how cool ...
1,I can’t get over how good everything looks. Dr...
2,Really hoping that this can save DC's movie un...
3,U cant deny how good this looks.Now if they ca...
4,"From this trailer, I have a feeling that this ..."
...,...
275,I want to see this. It may be one of his most ...
276,wow thats very amazing. I can't wait to see.
277,Doctor Fate is why i'm watching
278,This looks fire. DC looks like they stepping t...


In [9]:
nltk.download('vader_lexicon')
sentiments = SentimentIntensityAnalyzer()
data1["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data1["Comment"]]
data1["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data1["Comment"]]
data1["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data1["Comment"]]
data1['Compound'] = [sentiments.polarity_scores(i)["compound"] for i in data1["Comment"]]
score = data1["Compound"].values
sentiment = []
for i in score:
    if i >= 0.05 :
        sentiment.append('Positive')
    elif i <= -0.05 :
        sentiment.append('Negative')
    else:
        sentiment.append('Neutral')
data1["Sentiment"] = sentiment
data1.head()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\bader\AppData\Roaming\nltk_data...


,Comment,Positive,Negative,Neutral,Compound,Sentiment
0,Love how Dr. Fate's design looks and how cool ...,0.384,0.000,0.616,0.8910,Positive
1,I can’t get over how good everything looks. Dr...,0.153,0.000,0.847,0.6801,Positive
2,Really hoping that this can save DC's movie un...,0.375,0.000,0.625,0.9216,Positive
3,U cant deny how good this looks.Now if they ca...,0.302,0.049,0.649,0.9262,Positive
4,"From this trailer, I have a feeling that this ...",0.131,0.000,0.869,0.4416,Positive


In [10]:
data2=data1.drop(['Positive','Negative','Neutral','Compound'],axis=1)
data2.head()

,Comment,Sentiment
0,Love how Dr. Fate's design looks and how cool ...,Positive
1,I can’t get over how good everything looks. Dr...,Positive
2,Really hoping that this can save DC's movie un...,Positive
3,U cant deny how good this looks.Now if they ca...,Positive
4,"From this trailer, I have a feeling that this ...",Positive


In [14]:
stop_words = stopwords.words('english')
porter_stemmer = PorterStemmer()
lancaster_stemmer = LancasterStemmer() 
snowball_stemer = SnowballStemmer(language="english")
lzr = WordNetLemmatizer()

In [15]:
def text_processing(text):   
    # convert text into lowercase
    text = text.lower()

    # remove new line characters in text
    text = re.sub(r'\n',' ', text)
    
    # remove punctuations from text
    text = re.sub('[%s]' % re.escape(punctuation), "", text)
    
    # remove references and hashtags from text
    text = re.sub("^a-zA-Z0-9$,.", "", text)
    
    # remove multiple spaces from text
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    
    # remove special characters from text
    text = re.sub(r'\W', ' ', text)

    text = ' '.join([word for word in word_tokenize(text) if word not in stop_words])
    
    # stemming using porter stemmer from nltk package - msh a7sn 7aga - momken: lancaster, snowball
    # text=' '.join([porter_stemmer.stem(word) for word in word_tokenize(text)])
    # text=' '.join([lancaster_stemmer.stem(word) for word in word_tokenize(text)])
    # text=' '.join([snowball_stemer.stem(word) for word in word_tokenize(text)])
    
    # lemmatizer using WordNetLemmatizer from nltk package
    text=' '.join([lzr.lemmatize(word) for word in word_tokenize(text)])

    return text

In [18]:
nltk.download('omw-1.4')
data_copy = data2.copy()
data_copy.Comment = data_copy.Comment.apply(lambda text: text_processing(text))

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\bader\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\bader/nltk_data'
    - 'c:\\Users\\bader\\AppData\\Local\\anaconda3\\envs\\venv\\nltk_data'
    - 'c:\\Users\\bader\\AppData\\Local\\anaconda3\\envs\\venv\\share\\nltk_data'
    - 'c:\\Users\\bader\\AppData\\Local\\anaconda3\\envs\\venv\\lib\\nltk_data'
    - 'C:\\Users\\bader\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [19]:
le = LabelEncoder()
data_copy['Sentiment'] = le.fit_transform(data_copy['Sentiment'])

In [20]:
processed_data = {
    'Sentence':data_copy.Comment,
    'Sentiment':data_copy['Sentiment']
}

processed_data = pd.DataFrame(processed_data)
processed_data.head()

,Sentence,Sentiment
0,Love how Dr. Fate's design looks and how cool ...,2
1,I can’t get over how good everything looks. Dr...,2
2,Really hoping that this can save DC's movie un...,2
3,U cant deny how good this looks.Now if they ca...,2
4,"From this trailer, I have a feeling that this ...",2


In [21]:
processed_data['Sentiment'].value_counts()

Sentiment
2    205
1     39
0     36
Name: count, dtype: int64

In [22]:
df_neutral = processed_data[(processed_data['Sentiment']==1)] 
df_negative = processed_data[(processed_data['Sentiment']==0)]
df_positive = processed_data[(processed_data['Sentiment']==2)]

# upsample minority classes
df_negative_upsampled = resample(df_negative, 
                                 replace=True,    
                                 n_samples= 205, 
                                 random_state=42)  

df_neutral_upsampled = resample(df_neutral, 
                                 replace=True,    
                                 n_samples= 205, 
                                 random_state=42)  


# Concatenate the upsampled dataframes with the neutral dataframe
final_data = pd.concat([df_negative_upsampled,df_neutral_upsampled,df_positive])

In [23]:
final_data['Sentiment'].value_counts()

Sentiment
0    205
1    205
2    205
Name: count, dtype: int64

In [24]:
corpus = []
for sentence in final_data['Sentence']:
    corpus.append(sentence)
corpus[0:5]

["The trailer looks sick I'm definitely watching this movie",
 'He actually looks like the villain in this trailer!!!',
 'This movie is going to push DC to the top of comic book movies (for now)\r\nAfter the disaster that was the early DCEU, the new M-She-U mess and two awesome projekts behind them (The Suicide Squad and Peacemaker) yeah, this is going to rock (pun kinda intended)',
 "Damn this for sure I'm gonna watch...DC seems to be going on the right track now.",
 'okay… this looks absolutely incredible. DC making me look foolish for ever even being skeptical about this film. definitely seeing this opening weekend']

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500)
X = cv.fit_transform(corpus).toarray()
y = final_data.iloc[:, -1].values

Machine learning model

In [26]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

Evaluation

In [27]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm

array([[58,  0,  0],
       [ 0, 70,  0],
       [ 9,  1, 47]], dtype=int64)

In [28]:
nb_score = accuracy_score(y_test, y_pred)
print('accuracy',nb_score)

accuracy 0.9459459459459459


In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

# Diviser l'ensemble de données en un ensemble d'apprentissage et un ensemble de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Définir l'architecture du réseau de neurones
model = Sequential()
model.add(Dense(10, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compiler le modèle avec une fonction de perte et un optimiseur
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Former le modèle sur l'ensemble d'apprentissage
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Évaluer les performances du modèle sur l'ensemble de test
loss, accuracy = model.evaluate(X_test, y_test)
print('Exactitude: %.2f' % (accuracy*100))


Epoch 1/10
14/14 [==============================] - 2s 5ms/step - loss: 0.6203 - accuracy: 0.2930
Epoch 2/10
14/14 [==============================] - 0s 5ms/step - loss: 0.4682 - accuracy: 0.3302
Epoch 3/10
14/14 [==============================] - 0s 5ms/step - loss: 0.3102 - accuracy: 0.3558
Epoch 4/10
14/14 [==============================] - 0s 5ms/step - loss: 0.1371 - accuracy: 0.3837
Epoch 5/10
14/14 [==============================] - 0s 5ms/step - loss: -0.0431 - accuracy: 0.3860
Epoch 6/10
14/14 [==============================] - 0s 5ms/step - loss: -0.2268 - accuracy: 0.4116
Epoch 7/10
14/14 [==============================] - 0s 5ms/step - loss: -0.4071 - accuracy: 0.4163
Epoch 8/10
14/14 [==============================] - 0s 5ms/step - loss: -0.6044 - accuracy: 0.4163
Epoch 9/10
14/14 [==============================] - 0s 5ms/step - loss: -0.7976 - accuracy: 0.4233
Epoch 10/10
6/6 [==============================] - 1s 4ms/step - loss: -0.7725 - accuracy: 0.4649
Exactitude: 46.